**TODO**
- Sentiment analysis column
- Try out some models 

If all do bad go to next step and try word2vec:
- Word2Vec possibly on text column

In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import os
import seaborn as sns
import sys
import nltk



# makes plots nice
def make_pretty(ax, title='', x_label='', y_label='', img=False):
    ax.set_title(title, fontsize=20)
    ax.set_xlabel(x_label, fontsize=16)
    ax.set_ylabel(y_label, fontsize=16)
    ax.legend(loc='best', fontsize=16)
    if img:
        ax.axis('off')
    return ax

pd.set_option('display.max_rows', 250)
pd.set_option('display.max_columns', 250)


data_path = os.path.join('nice_combined_data.csv')

In [2]:
df = pd.read_csv(data_path)

In [3]:
df['target'] = df['target'].apply(lambda x : 1 if x > 0 else 0)
df['Date'] = pd.to_datetime(df['Date'])

In [4]:
df.head()

,Date,mentioned,text,mentioned_count,upvote_ratio,score,gilded,total_awards_received,num_comments,target
0,2021-01-04,AAPL,weekend iv report ticker low iv cheap premium ...,14,0.763571,189.357143,0.428571,5.214286,94.357143,0
1,2021-01-04,BA,weekend iv report ticker low iv cheap premium ...,5,0.782000,131.000000,0.000000,0.200000,32.800000,0
2,2021-01-04,BABA,weekend iv report ticker low iv cheap premium ...,8,0.728750,45.250000,0.000000,0.125000,20.750000,1
3,2021-01-04,BB,guess company due diligence without revealing ...,1,0.960000,417.000000,1.000000,2.000000,51.000000,0
4,2021-01-04,CMCSA,lesson 2020 x200bhi debated make post thought ...,1,0.920000,304.000000,0.000000,0.000000,61.000000,0


In [6]:
from nltk.sentiment import SentimentAnalyzer
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')


analyzer = SentimentIntensityAnalyzer()

df['polarity'] = df['text'].apply(lambda x : analyzer.polarity_scores(x))



[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\paganinik\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [7]:
df['polarity']

0       {'neg': 0.065, 'neu': 0.768, 'pos': 0.167, 'co...
1       {'neg': 0.077, 'neu': 0.753, 'pos': 0.171, 'co...
2       {'neg': 0.047, 'neu': 0.803, 'pos': 0.15, 'com...
3       {'neg': 0.067, 'neu': 0.841, 'pos': 0.092, 'co...
4       {'neg': 0.093, 'neu': 0.715, 'pos': 0.192, 'co...
                              ...                        
6562    {'neg': 0.036, 'neu': 0.743, 'pos': 0.221, 'co...
6563    {'neg': 0.069, 'neu': 0.731, 'pos': 0.199, 'co...
6564    {'neg': 0.072, 'neu': 0.733, 'pos': 0.195, 'co...
6565    {'neg': 0.007, 'neu': 0.745, 'pos': 0.248, 'co...
6566    {'neg': 0.064, 'neu': 0.826, 'pos': 0.11, 'com...
Name: polarity, Length: 6567, dtype: object

,Date,mentioned,text,mentioned_count,upvote_ratio,score,gilded,total_awards_received,num_comments,target,neg,neu,pos,compound
0,2021-01-04,AAPL,weekend iv report ticker low iv cheap premium ...,14,0.763571,189.357143,0.428571,5.214286,94.357143,0,0.065,0.768,0.167,0.9999
1,2021-01-04,BA,weekend iv report ticker low iv cheap premium ...,5,0.782000,131.000000,0.000000,0.200000,32.800000,0,0.077,0.753,0.171,0.9998
3,2021-01-04,BB,guess company due diligence without revealing ...,1,0.960000,417.000000,1.000000,2.000000,51.000000,0,0.067,0.841,0.092,0.6963
4,2021-01-04,CMCSA,lesson 2020 x200bhi debated make post thought ...,1,0.920000,304.000000,0.000000,0.000000,61.000000,0,0.093,0.715,0.192,0.9959
5,2021-01-04,CSCO,weekend iv report ticker low iv cheap premium ...,3,0.750000,49.666667,0.000000,0.666667,12.666667,0,0.028,0.812,0.161,1.0000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6557,2021-12-30,NVDA,red year sp500 30 lately keep hearing old narr...,4,0.852500,589.750000,0.000000,2.500000,163.000000,0,0.065,0.617,0.318,0.9997
6558,2021-12-30,PEP,stock news interglobe aviation rbl bank sigach...,1,0.670000,1.000000,0.000000,0.000000,1.000000,0,0.011,0.876,0.113,0.9919
6564,2021-12-30,V,fundamental valuation model ethereum long didn...,3,0.860000,8.000000,0.000000,0.333333,25.666667,0,0.072,0.733,0.195,0.9994
6565,2021-12-30,WFC,checking account recs 5k atm friendly im hopin...,2,0.880000,2.000000,0.000000,0.000000,4.500000,0,0.007,0.745,0.248,0.9902
